In [27]:
#Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-developer-guide/blob/master/LICENSE-SAMPLECODE.)

import boto3
import json
import sys
import time



class VideoDetect:
    jobId = ''
    rek = boto3.client('rekognition')
    sqs = boto3.client('sqs')
    sns = boto3.client('sns')
    
    roleArn = ''
    bucket = ''
    video = ''
    startJobId = ''

    sqsQueueUrl = ''
    snsTopicArn = ''
    processType = ''

    def __init__(self, role, bucket, video):    
        self.roleArn = role
        self.bucket = bucket
        self.video = video

    def GetSQSMessageSuccess(self):

        jobFound = False
        succeeded = False
    
        dotLine=0
        while jobFound == False:
            sqsResponse = self.sqs.receive_message(QueueUrl=self.sqsQueueUrl, MessageAttributeNames=['ALL'],
                                          MaxNumberOfMessages=10)

            if sqsResponse:
                
                if 'Messages' not in sqsResponse:
                    if dotLine<40:
                        print('.', end='')
                        dotLine=dotLine+1
                    else:
                        print()
                        dotLine=0    
                    sys.stdout.flush()
                    time.sleep(5)
                    continue

                for message in sqsResponse['Messages']:
                    notification = json.loads(message['Body'])
                    rekMessage = json.loads(notification['Message'])
                    print(rekMessage['JobId'])
                    print(rekMessage['Status'])
                    if rekMessage['JobId'] == self.startJobId:
                        print('Matching Job Found:' + rekMessage['JobId'])
                        jobFound = True
                        if (rekMessage['Status']=='SUCCEEDED'):
                            succeeded=True

                        self.sqs.delete_message(QueueUrl=self.sqsQueueUrl,
                                       ReceiptHandle=message['ReceiptHandle'])
                    else:
                        print("Job didn't match:" +
                              str(rekMessage['JobId']) + ' : ' + self.startJobId)
                    # Delete the unknown message. Consider sending to dead letter queue
                    self.sqs.delete_message(QueueUrl=self.sqsQueueUrl,
                                   ReceiptHandle=message['ReceiptHandle'])


        return succeeded

    def StartFaceDetection(self):
        response=self.rek.start_face_detection(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
            NotificationChannel={'RoleArn': self.roleArn, 'SNSTopicArn': self.snsTopicArn},FaceAttributes='ALL')

        self.startJobId=response['JobId']
        print('Start Job Id: ' + self.startJobId)

    def GetFaceDetectionResults(self):
        maxResults = 10
        paginationToken = ''
        finished = False

        while finished == False:
            response = self.rek.get_face_detection(JobId=self.startJobId,
                                            MaxResults=maxResults,
                                            NextToken=paginationToken)

            print('Codec: ' + response['VideoMetadata']['Codec'])
            print('Duration: ' + str(response['VideoMetadata']['DurationMillis']))
            print('Format: ' + response['VideoMetadata']['Format'])
            print('Frame rate: ' + str(response['VideoMetadata']['FrameRate']))
            print()
            
            for faceDetection in response['Faces']:
                print('Face:' + str(faceDetection['Face']))
                print('Timestamp: ' + str(faceDetection['Timestamp']))
                print()

            if 'NextToken' in response:
                paginationToken = response['NextToken']
            else:
                finished = True
                    
                    
    def CreateTopicandQueue(self):
      
        millis = str(int(round(time.time() * 1000)))

        #Create SNS topic
        
        snsTopicName="AmazonRekognitionExample" + millis

        topicResponse=self.sns.create_topic(Name=snsTopicName)
        self.snsTopicArn = topicResponse['TopicArn']

        #create SQS queue
        sqsQueueName="AmazonRekognitionQueue" + millis
        self.sqs.create_queue(QueueName=sqsQueueName)
        self.sqsQueueUrl = self.sqs.get_queue_url(QueueName=sqsQueueName)['QueueUrl']
 
        attribs = self.sqs.get_queue_attributes(QueueUrl=self.sqsQueueUrl,
                                                    AttributeNames=['QueueArn'])['Attributes']
                                        
        sqsQueueArn = attribs['QueueArn']

        # Subscribe SQS queue to SNS topic
        self.sns.subscribe(
            TopicArn=self.snsTopicArn,
            Protocol='sqs',
            Endpoint=sqsQueueArn)

        #Authorize SNS to write SQS queue 
        policy = """{{
        "Version":"2012-10-17",
        "Statement":[
         {{
           "Sid":"MyPolicy",
           "Effect":"Allow",
           "Principal" : {{"AWS" : "*"}},
           "Action":"SQS:SendMessage",
           "Resource": "{}",
           "Condition":{{
           "ArnEquals":{{
           "aws:SourceArn": "{}"
            }}
          }}
        }}
       ]
       }}""".format(sqsQueueArn, self.snsTopicArn)
 
        response = self.sqs.set_queue_attributes(
            QueueUrl = self.sqsQueueUrl,
            Attributes = {
                'Policy' : policy
            })

    def DeleteTopicandQueue(self):
        self.sqs.delete_queue(QueueUrl=self.sqsQueueUrl)
        self.sns.delete_topic(TopicArn=self.snsTopicArn)


def main():
    roleArn = 'arn:aws:iam::404275428206:role/my_role'   
    bucket = 'videoanabucket'
    video = 'new_video.mp4'

    analyzer=VideoDetect(roleArn, bucket,video)
    analyzer.CreateTopicandQueue()
    

    analyzer.StartFaceDetection()
    if analyzer.GetSQSMessageSuccess()==True:
        analyzer.GetFaceDetectionResults()
        
    analyzer.DeleteTopicandQueue()


if __name__ == "__main__":
    main()

Start Job Id: d26a5a4e4bcd4b0d5d7fee41998552cc084cb0fa37d441807e1f1f1e4e87b40e
..................d26a5a4e4bcd4b0d5d7fee41998552cc084cb0fa37d441807e1f1f1e4e87b40e
SUCCEEDED
Matching Job Found:d26a5a4e4bcd4b0d5d7fee41998552cc084cb0fa37d441807e1f1f1e4e87b40e
Codec: h264
Duration: 59833
Format: QuickTime / MOV
Frame rate: 30.0

Face:{'BoundingBox': {'Width': 0.1305820196866989, 'Height': 0.35249239206314087, 'Left': 0.4255248010158539, 'Top': 0.11717253923416138}, 'AgeRange': {'Low': 29, 'High': 45}, 'Smile': {'Value': True, 'Confidence': 94.48235321044922}, 'Eyeglasses': {'Value': False, 'Confidence': 99.88813781738281}, 'Sunglasses': {'Value': False, 'Confidence': 99.96805572509766}, 'Gender': {'Value': 'Male', 'Confidence': 99.93045043945312}, 'Beard': {'Value': True, 'Confidence': 95.7323226928711}, 'Mustache': {'Value': False, 'Confidence': 89.0517807006836}, 'EyesOpen': {'Value': False, 'Confidence': 51.13261413574219}, 'MouthOpen': {'Value': True, 'Confidence': 97.66890716552734}, '

Codec: h264
Duration: 59833
Format: QuickTime / MOV
Frame rate: 30.0

Face:{'BoundingBox': {'Width': 0.14037710428237915, 'Height': 0.38282713294029236, 'Left': 0.4330778121948242, 'Top': 0.1141146793961525}, 'AgeRange': {'Low': 32, 'High': 48}, 'Smile': {'Value': False, 'Confidence': 99.8629379272461}, 'Eyeglasses': {'Value': False, 'Confidence': 99.91578674316406}, 'Sunglasses': {'Value': False, 'Confidence': 99.9692611694336}, 'Gender': {'Value': 'Male', 'Confidence': 99.90443420410156}, 'Beard': {'Value': True, 'Confidence': 93.84503936767578}, 'Mustache': {'Value': False, 'Confidence': 90.53052520751953}, 'EyesOpen': {'Value': True, 'Confidence': 70.22240447998047}, 'MouthOpen': {'Value': False, 'Confidence': 81.8340835571289}, 'Emotions': [{'Type': 'CALM', 'Confidence': 84.77944946289062}, {'Type': 'CONFUSED', 'Confidence': 7.085023880004883}, {'Type': 'SAD', 'Confidence': 5.5441694259643555}, {'Type': 'FEAR', 'Confidence': 1.3084452152252197}, {'Type': 'SURPRISED', 'Confidence':

Codec: h264
Duration: 59833
Format: QuickTime / MOV
Frame rate: 30.0

Face:{'BoundingBox': {'Width': 0.1444629728794098, 'Height': 0.41000187397003174, 'Left': 0.4150690734386444, 'Top': 0.12310776859521866}, 'AgeRange': {'Low': 29, 'High': 45}, 'Smile': {'Value': False, 'Confidence': 71.29054260253906}, 'Eyeglasses': {'Value': False, 'Confidence': 93.33570098876953}, 'Sunglasses': {'Value': False, 'Confidence': 97.20343017578125}, 'Gender': {'Value': 'Male', 'Confidence': 99.82575988769531}, 'Beard': {'Value': True, 'Confidence': 90.32367706298828}, 'Mustache': {'Value': False, 'Confidence': 92.83229064941406}, 'EyesOpen': {'Value': True, 'Confidence': 99.12252044677734}, 'MouthOpen': {'Value': True, 'Confidence': 64.89361572265625}, 'Emotions': [{'Type': 'HAPPY', 'Confidence': 64.2300033569336}, {'Type': 'CALM', 'Confidence': 26.198991775512695}, {'Type': 'SURPRISED', 'Confidence': 2.5131964683532715}, {'Type': 'FEAR', 'Confidence': 2.468245029449463}, {'Type': 'CONFUSED', 'Confidenc

Codec: h264
Duration: 59833
Format: QuickTime / MOV
Frame rate: 30.0

Face:{'BoundingBox': {'Width': 0.14199867844581604, 'Height': 0.3658273220062256, 'Left': 0.4267789125442505, 'Top': 0.1290581226348877}, 'AgeRange': {'Low': 32, 'High': 48}, 'Smile': {'Value': False, 'Confidence': 92.9189453125}, 'Eyeglasses': {'Value': False, 'Confidence': 99.85425567626953}, 'Sunglasses': {'Value': False, 'Confidence': 99.94491577148438}, 'Gender': {'Value': 'Male', 'Confidence': 99.94893646240234}, 'Beard': {'Value': True, 'Confidence': 95.8653564453125}, 'Mustache': {'Value': False, 'Confidence': 91.50723266601562}, 'EyesOpen': {'Value': True, 'Confidence': 75.37810516357422}, 'MouthOpen': {'Value': False, 'Confidence': 59.39601516723633}, 'Emotions': [{'Type': 'CALM', 'Confidence': 79.39803314208984}, {'Type': 'HAPPY', 'Confidence': 14.822114944458008}, {'Type': 'CONFUSED', 'Confidence': 2.0367014408111572}, {'Type': 'SAD', 'Confidence': 1.6720808744430542}, {'Type': 'ANGRY', 'Confidence': 0.83

Codec: h264
Duration: 59833
Format: QuickTime / MOV
Frame rate: 30.0

Face:{'BoundingBox': {'Width': 0.13520288467407227, 'Height': 0.38192155957221985, 'Left': 0.4048348367214203, 'Top': 0.13590173423290253}, 'AgeRange': {'Low': 26, 'High': 42}, 'Smile': {'Value': False, 'Confidence': 96.1569595336914}, 'Eyeglasses': {'Value': False, 'Confidence': 98.80120849609375}, 'Sunglasses': {'Value': False, 'Confidence': 99.49440002441406}, 'Gender': {'Value': 'Male', 'Confidence': 99.8167724609375}, 'Beard': {'Value': True, 'Confidence': 92.35576629638672}, 'Mustache': {'Value': False, 'Confidence': 92.2227783203125}, 'EyesOpen': {'Value': True, 'Confidence': 97.97383880615234}, 'MouthOpen': {'Value': True, 'Confidence': 69.96822357177734}, 'Emotions': [{'Type': 'FEAR', 'Confidence': 43.78870391845703}, {'Type': 'SURPRISED', 'Confidence': 23.748395919799805}, {'Type': 'CALM', 'Confidence': 19.21466064453125}, {'Type': 'HAPPY', 'Confidence': 5.773715496063232}, {'Type': 'CONFUSED', 'Confidence'

Codec: h264
Duration: 59833
Format: QuickTime / MOV
Frame rate: 30.0

Face:{'BoundingBox': {'Width': 0.13141489028930664, 'Height': 0.3481019139289856, 'Left': 0.41442251205444336, 'Top': 0.1318056434392929}, 'AgeRange': {'Low': 31, 'High': 47}, 'Smile': {'Value': False, 'Confidence': 99.85783386230469}, 'Eyeglasses': {'Value': False, 'Confidence': 99.88133239746094}, 'Sunglasses': {'Value': False, 'Confidence': 99.94976043701172}, 'Gender': {'Value': 'Male', 'Confidence': 99.72570037841797}, 'Beard': {'Value': True, 'Confidence': 93.55560302734375}, 'Mustache': {'Value': False, 'Confidence': 91.2354736328125}, 'EyesOpen': {'Value': False, 'Confidence': 57.91576385498047}, 'MouthOpen': {'Value': False, 'Confidence': 92.9477310180664}, 'Emotions': [{'Type': 'CALM', 'Confidence': 89.77776336669922}, {'Type': 'CONFUSED', 'Confidence': 4.68123197555542}, {'Type': 'SAD', 'Confidence': 3.783872604370117}, {'Type': 'FEAR', 'Confidence': 0.8952993750572205}, {'Type': 'ANGRY', 'Confidence': 0.5

Codec: h264
Duration: 59833
Format: QuickTime / MOV
Frame rate: 30.0

Face:{'BoundingBox': {'Width': 0.13535207509994507, 'Height': 0.3762781322002411, 'Left': 0.4056355059146881, 'Top': 0.1381365805864334}, 'AgeRange': {'Low': 29, 'High': 45}, 'Smile': {'Value': False, 'Confidence': 99.61014556884766}, 'Eyeglasses': {'Value': False, 'Confidence': 99.3639907836914}, 'Sunglasses': {'Value': False, 'Confidence': 99.71173095703125}, 'Gender': {'Value': 'Male', 'Confidence': 99.81304931640625}, 'Beard': {'Value': True, 'Confidence': 93.8064193725586}, 'Mustache': {'Value': False, 'Confidence': 89.33094787597656}, 'EyesOpen': {'Value': True, 'Confidence': 96.36552429199219}, 'MouthOpen': {'Value': False, 'Confidence': 56.91239929199219}, 'Emotions': [{'Type': 'CALM', 'Confidence': 55.086368560791016}, {'Type': 'FEAR', 'Confidence': 18.690900802612305}, {'Type': 'CONFUSED', 'Confidence': 11.865426063537598}, {'Type': 'SURPRISED', 'Confidence': 9.151844024658203}, {'Type': 'SAD', 'Confidence'